In [1]:
import pandas as pd
import torch 
from transformers import T5ForConditionalGeneration, T5Tokenizer
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import GPT2LMHeadModel, GPT2Tokenizer
from sklearn.model_selection import train_test_split
from datetime import datetime as dt
import import_ipynb
from sari import SARIsent
from datasets import load_dataset, load_metric
from rouge_score import rouge_scorer
import numpy as  np

C:\Users\nkhozin\anaconda3\envs\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


importing Jupyter notebook from sari.ipynb


In [21]:
device = 'cuda'

In [22]:
def add_special_tokens(model_name):
    tokenizer = GPT2Tokenizer.from_pretrained(model_name)
    special_tokens = {'pad_token':'<|pad|>','sep_token':'<|sep|>'}
    num_add_toks = tokenizer.add_special_tokens(special_tokens)
    tokenizer.src_lang = 'ru'
    return tokenizer

def generate_t5(model, tokenizer, text):
    inputs = tokenizer(text, truncation=True, return_tensors='pt').to(device)
    with torch.no_grad():
        hypotheses = model.generate(inputs["input_ids"], max_length=len(text)*0.7) #, max_length=30, num_beams=5
    return tokenizer.decode(hypotheses[0], skip_special_tokens=True)

def generate_bart(model, tokenizer, text):
    model_inputs = tokenizer(text, padding='max_length', truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        generated_ids = model.generate(model_inputs["input_ids"], max_length=int(len(text)*0.7), do_sample=False) #, max_new_tokens=100 , max_new_tokens=100
    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

def generate_gpt(model, tokenizer, text):
    model_inputs = tokenizer(text, padding='max_length', truncation=True, return_tensors="pt").to(device)
    with torch.no_grad():
        generated_ids = model.generate(model_inputs["input_ids"], max_length=int(len(text)*0.7), do_sample=False, pad_token_id=tokenizer.eos_token_id) #, max_new_tokens=100 , max_new_tokens=100
    return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]

In [2]:
test = pd.read_csv('..\\data\\test_with_pred.csv', index_col=0)

In [26]:
group = test.groupby(['type','size'], as_index=False).agg({
    'source_len':['count','min','max','mean'],
    'target_len':['min','max','mean']
    })
round(group)

type   size source_len                 target_len           
                            count min  max   mean        min  max  mean
0  para_phraser  small        640  12   73   25.0          5   32  15.0
1      ru_adapt  small        640  10  392   56.0          5  297  37.0
2       ru_simp  small        640  26  157   42.0          6   93  26.0
3      ru_xlsum  small        640  51  400  234.0         14  123  44.0

In [27]:
test.shape

(2560, 70)

In [28]:
test.head(3)

,source,target,type,source_len,target_len,size,part,mark,rut5_base_pred,rut5_base_time_sec,...,rougeL_rut5_v5,rouge1_rut5_v6,rouge2_rut5_v6,rougeL_rut5_v6,rouge1_rut5_v7,rouge2_rut5_v7,rougeL_rut5_v7,rouge1_target,rouge2_target,rougeL_target
8654,simplify | Стереотипное и повторяющееся моторн...,поведение определяет диагностику при синдроме ...,ru_simp,45,12,small,26.666667,1,Стереотипное и повторяющееся моторное поведени...,485.410721,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
762,simplify | В Ташкентской и Навоийской областях...,В некоторых областях Узбекистана работают шк...,ru_simp,45,29,small,64.444444,1,В Ташкентской и Навоийской областях функционир...,485.410721,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10246,"simplify | Отправляя мессу в городе Трухильйо,...",Предстоятель Римско-католической церкви Папа Ф...,ru_xlsum,150,41,small,27.333333,1,"Папа назвал насилие ""чумой"", с которой надо бо...",485.410721,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
def make_preditiction(data, models_name):
    path = '..//models//'
    for model_name in models_name:
        print(model_name)
        start_time = dt.now()
        if model_name=='bart':
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model = AutoModelForSeq2SeqLM.from_pretrained(model_name).cuda()
            data[f"{model_name}_pred"] = data.source.apply(lambda value: generate_bart(model, tokenizer, value))
        elif model_name=='rugpt3':
            tokenizer_name = 'sberbank-ai/rugpt3small_based_on_gpt2'
            tokenizer = add_special_tokens(tokenizer_name)
            model = GPT2LMHeadModel.from_pretrained(model_name, local_files_only=True).cuda()
            data[f"{model_name}_pred"] = data.source.apply(lambda value: generate_gpt(model, tokenizer, value))
        else:
            model = T5ForConditionalGeneration.from_pretrained(path+model_name).cuda()
            tokenizer = T5Tokenizer.from_pretrained(path+model_name)
            if model_name=="cointegrated/rut5-base-multitask":
                model_name = "rut5_base"
            data[f"{model_name}_pred"] = data.source.apply(lambda value: generate_t5(model, tokenizer, value))
        end_time = dt.now()
        data[f'{model_name}_time_sec'] = (end_time-start_time).total_seconds()
    return data 

def calculate_sari(data, models_name):
    for model_name in models_name:
        if model_name=="cointegrated/rut5-base-multitask":
            model_name = "rut5_base"
        data[f"SARI_{model_name}"] = data.apply(lambda row: SARIsent(row['source'], row[f"{model_name}_pred"] ,row['target']), axis=1)*100
    return data

def calculate_rouge_f1(predict, target, metric):
    scorer = rouge_scorer.RougeScorer([metric])
    scores = scorer.score(predict,target)[metric]
    precision = scores.precision*100
    recall = scores.recall*100
    try:
        f1_score = (2*precision*recall)/(precision+recall)
    except:
        f1_score = 0
    return f1_score

def calculate_rouge(data, models_name):
    for model_name in models_name:
        if model_name=="cointegrated/rut5-base-multitask":
            model_name = "rut5_base"
        data[f"rouge1_{model_name}"] = data.apply(lambda row: calculate_rouge_f1(row[f"{model_name}_pred"] ,row['target'], 'rouge1'), axis=1)
        data[f"rouge2_{model_name}"] = data.apply(lambda row: calculate_rouge_f1(row[f"{model_name}_pred"] ,row['target'], 'rouge2'), axis=1)
        data[f"rougeL_{model_name}"] = data.apply(lambda row: calculate_rouge_f1(row[f"{model_name}_pred"] ,row['target'], 'rougeL'), axis=1)
    return data

In [41]:
models_name = [
    "cointegrated/rut5-base-multitask",
    "bart",
    "rut5_v1",
    "rut5_v2",
    "rut5_v3",
    "rut5_v4",
    "rut5_v5",
    "rut5_v6",
    "rut5_v7",
    "rut5_v8"
]

models_desc= {
    "cointegrated/rut5-base-multitask":['baseline',6,3],
    "bart":['train(small-12k pairds)',6,3],
    "rugpt3":['train(small-12k pairds)',6,3],
    "rut5_v1":['train(small-12k pairds)',6,3],
    "rut5_v2":['ru_adapt_small(small)',6,3],
    "rut5_v3":['ru_xlsum_small(small)',6,3],
    "rut5_v4":['mix(medium)',3,3],
    "rut5_v5":['mix(large)',1,3],
    "rut5_v6":['para_phraser(small)',6,1],
    "rut5_v7":['rut5_v2+train_all(small+medium)',3,1],
    "rut5_v8":['base+train_all_clean(small+medium)',3,1]
}

In [22]:
%%time

test = make_preditiction(test, models_name)

cointegrated/rut5-base-multitask


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


bart
rugpt3


Asking to pad to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no padding.
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:5025

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable resul

...

rut5_v6


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


rut5_v7


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


CPU times: total: 2h 35min 8s
Wall time: 2h 35min 29s


In [40]:
%%time

test = calculate_sari(test, models_name)

CPU times: total: 9.91 s
Wall time: 9.9 s


In [41]:
%%time

test = calculate_rouge(test, models_name)

CPU times: total: 1.38 s
Wall time: 1.36 s


In [31]:
models_name = [
    "rut5_v8"
]

models_desc= {
    "rut5_v8":['base+train_all_clean(small+medium)',3,1]
}

In [32]:
%%time

test = make_preditiction(test, models_name)
test = calculate_sari(test, models_name)
test = calculate_rouge(test, models_name)

rut5_v8


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


CPU times: total: 12min 11s
Wall time: 12min 10s


In [33]:
show = test.mean().reset_index()
show = show[show['index'].str.contains('SARI')]
show

C:\Users\nkhozin\AppData\Local\Temp\ipykernel_17240\1138905521.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  show = test.mean().reset_index()


,index,0
6,SARI_rut5_base,33.354310
7,SARI_bart,33.495816
22,SARI_rut5_v1,33.429701
23,SARI_rut5_v2,33.426770
24,SARI_rut5_v3,33.590926
25,SARI_rut5_v4,33.574417
26,SARI_rut5_v5,33.657694
27,SARI_rut5_v6,33.623879
28,SARI_rut5_v7,33.398022
57,SARI_rut5_v8,33.415377


In [34]:
show = test.mean().reset_index()
show = show[show['index'].str.contains('rouge1')]
show

C:\Users\nkhozin\AppData\Local\Temp\ipykernel_17240\2864469106.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  show = test.mean().reset_index()


,index,0
8,rouge1_rut5_base,6.101911
11,rouge1_bart,5.323444
29,rouge1_rugpt3,5.658854
32,rouge1_rut5_v1,6.573093
35,rouge1_rut5_v2,6.677325
38,rouge1_rut5_v3,4.612165
41,rouge1_rut5_v4,5.466009
44,rouge1_rut5_v5,3.680313
47,rouge1_rut5_v6,2.388672
50,rouge1_rut5_v7,7.219897


In [35]:
show = test.mean().reset_index()
show = show[show['index'].str.contains('rouge2')]
show['index'] = show['index'].replace('rouge2_','')
show

C:\Users\nkhozin\AppData\Local\Temp\ipykernel_17240\417953610.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  show = test.mean().reset_index()


,index,0
9,rouge2_rut5_base,1.473958
12,rouge2_bart,1.276042
30,rouge2_rugpt3,1.720632
33,rouge2_rut5_v1,2.136039
36,rouge2_rut5_v2,1.937593
39,rouge2_rut5_v3,1.070312
42,rouge2_rut5_v4,1.516743
45,rouge2_rut5_v5,0.621869
48,rouge2_rut5_v6,0.247396
51,rouge2_rut5_v7,2.234189


In [36]:
show = test.mean().reset_index()
show = show[show['index'].str.contains('rougeL')]
show

C:\Users\nkhozin\AppData\Local\Temp\ipykernel_17240\2128926639.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  show = test.mean().reset_index()


,index,0
10,rougeL_rut5_base,6.066104
13,rougeL_bart,5.310423
31,rougeL_rugpt3,5.599983
34,rougeL_rut5_v1,6.516360
37,rougeL_rut5_v2,6.621987
40,rougeL_rut5_v3,4.577009
43,rougeL_rut5_v4,5.433457
46,rougeL_rut5_v5,3.600235
49,rougeL_rut5_v6,2.388672
52,rougeL_rut5_v7,7.150143


In [55]:
#Rouge по парам source, target

test["rouge1_target"] = test.apply(lambda row: calculate_rouge_f1(row["target"], row['source'], 'rouge1'), axis=1)
test["rouge2_target"] = test.apply(lambda row: calculate_rouge_f1(row["target"], row['source'], 'rouge2'), axis=1)
test["rougeL_target"] = test.apply(lambda row: calculate_rouge_f1(row["target"], row['source'], 'rougeL'), axis=1)

In [72]:
test.groupby('type', as_index=False).agg({
    'rouge1_target':['mean','max'],
    'rouge2_target':['mean','max'],
    'rougeL_target':['mean','max'],
})

type rouge1_target            rouge2_target             \
                         mean        max          mean        max   
0  para_phraser      7.170132  85.714286      1.139595  66.666667   
1      ru_adapt      8.027974  94.736842      3.578920  94.117647   
2       ru_simp      3.126488  85.714286      1.036458  80.000000   
3      ru_xlsum      4.310822  80.000000      1.127555  66.666667   

  rougeL_target             
           mean        max  
0      7.018346  80.000000  
1      8.027974  94.736842  
2      3.126488  85.714286  
3      4.227124  80.000000

In [37]:
test.to_csv('..\\data\\test_with_pred.csv.csv')

In [42]:
col_names = ['model','dataset|batch_size|num_epochs','SARI','rouge1','rouge2','rougeL','time_sec']
result = pd.DataFrame(columns=col_names)

for model_name in models_name:
    desc = models_desc[model_name]
    if model_name=="cointegrated/rut5-base-multitask":
        model_name = "rut5_base"
    new_row = [model_name, str(desc)]
    for metric in [f"SARI_{model_name}", f"rouge1_{model_name}",f"rouge2_{model_name}", f"rougeL_{model_name}", f'{model_name}_time_sec']:
        new_row.append(round(test[metric].mean(),1))
    
    new_row_df = pd.DataFrame(np.array(new_row).reshape(-1,len(new_row)), columns=col_names)
    result = pd.concat([result, new_row_df], axis=0)

In [43]:
result['avg_time'] = result.time_sec.astype('float')/test.shape[0]

In [44]:
result

,model,dataset|batch_size|num_epochs,SARI,rouge1,rouge2,rougeL,time_sec,avg_time
0,rut5_base,"['baseline', 6, 3]",33.4,6.1,1.5,6.1,485.4,0.189609
0,bart,"['train(small-12k pairds)', 6, 3]",33.5,5.3,1.3,5.3,2048.0,0.800000
0,rut5_v1,"['train(small-12k pairds)', 6, 3]",33.4,6.6,2.1,6.5,602.4,0.235312
0,rut5_v2,"['ru_adapt_small(small)', 6, 3]",33.4,6.7,1.9,6.6,721.3,0.281758
0,rut5_v3,"['ru_xlsum_small(small)', 6, 3]",33.6,4.6,1.1,4.6,523.6,0.204531
0,rut5_v4,"['mix(medium)', 3, 3]",33.6,5.5,1.5,5.4,630.9,0.246445
0,rut5_v5,"['mix(large)', 1, 3]",33.7,3.7,0.6,3.6,668.9,0.261289
0,rut5_v6,"['para_phraser(small)', 6, 1]",33.6,2.4,0.2,2.4,386.9,0.151133
0,rut5_v7,"['rut5_v2+train_all(small+medium)', 3, 1]",33.4,7.2,2.2,7.2,657.3,0.256758
0,rut5_v8,"['base+train_all_clean(small+medium)', 3, 1]",33.4,7.1,2.0,7.0,729.2,0.284844
